### MCMF(최소 비용 최대 유량)
- 최소 비용을 이용하여 최대 유량을 흘리는 문제.
- 즉 기존의 유량 그래프 내의 간선에 비용 정보가 추가된 문제이다.
- 유량이 최대일 때 그 비용의 합을 최소로 하는 문제이다.

### SPFA(Shortest Path Faster Algorithm)
- Ford-Fulkerson, Edmond-Karp 알고리즘과 비슷하게 매번 경로를 찾아 더 이상의 경로가 없을 때 까지 유량을 흘리는데, 그 경로를 최단 경로로 찾는다.
  - 여기서 최단경로란, `Source에서 Sink로 가는 최소 비용의 거리` = 간선의 가중치가 비용이라고 생각하고 그 최소비용을 찾는다.
- 매번 최단 경로를 찾고, 그 경로를 통해 흐를 수 있는 최대 유량을 찾은 후 흘려보내면서, 그 결과에 이번에 찾은 경로의 `비용 합` * `유량`을 더해준다.
- 보통 최단 경로를 변형한 알고리즘을 사용하지만, 적어도 음의 가중치를 사용할 수 있어야 한다.
  - 따라서 Bellman-Ford를 변형한 SPFA를 사용한다.
  - 시간복잡도는 $O(VEf^*)$ 이다.
- 최소 비용으로 이분매칭을 하는 경우는 헝가리안 알고리즘을 사용한다고 한다.

### 구현
- 좀 많이 느린 포드포커슨 스타일 구현이 11408(열혈강호 5)에 있다. $1 \le V \le 800$ 의 범위에서 TLE 없이 문제를 풀 수 있었다.
- 현재 구현은 Dinic 스타일 최적화를 적용한 코드

In [ ]:
import collections
INF = 1 << 30
class MCMF:
  def __init__(self, V, src, snk) :
    self.len = V
    self.G = [[] for _ in range(V)]
    self.C = [[0] * V for _ in range(V)]
    self.F = [[0] * V for _ in range(V)]
    self.W = [[0] * V for _ in range(V)]
    self.chk = [False] * V
    self.vis = [False] * V
    self.lvl = [0] * V
    self.src = src
    self.snk = snk
    self.mc = 0

  def add(self, u, v, w, c) :
    self.G[u].append(v)
    self.G[v].append(u)
    self.W[u][v] = w
    self.W[v][u] = -w
    self.C[u][v] = c

  def bfs(self) :
    Q = collections.deque([self.src])
    inQ = [False] * self.len 
    lvl = [INF] * self.len
    lvl[self.src] = 0

    while Q :
      u = Q.popleft()
      inQ[u] = False

      for v in self.G[u] :
        if self.C[u][v] > self.F[u][v] and lvl[v] > lvl[u] + self.W[u][v] :
          lvl[v] = lvl[u] + self.W[u][v]
          if not inQ[v] :
            Q.append(v)
            inQ[v] = True
    
    self.lvl = lvl
    return self.lvl[self.snk] != INF

  def dfs(self, u, w) :
    if u == self.snk : return w
    flow = w
    self.vis[u] = True

    for v in self.G[u] :
      if not flow : break
      if not self.vis[v] and not self.chk[v] and self.C[u][v] - self.F[u][v] > 0 and self.lvl[v] == self.lvl[u] + self.W[u][v] :
        res = self.dfs(v, min(flow, self.C[u][v] - self.F[u][v]))
        if res == 0 : 
          self.chk[v] = True
          continue
        self.F[u][v] += res
        self.F[v][u] -= res
        self.mc += res * self.W[u][v]
        flow -= res
    
    self.vis[u] = False
    return w - flow

  def calc(self) :
    mf = 0
    while self.bfs() :
      self.chk = [False] * self.len
      self.vis = [False] * self.len
      mf += self.dfs(self.src, INF)
    return self.mc, mf

### TODO
- 인접 List SFPA?
- 인기가 없는 알고리즘인지 템플릿 구하기가 영 쉽지 않다
- https://www.teferi.net/ps/%EC%B5%9C%EC%86%8C_%EB%B9%84%EC%9A%A9_%EC%B5%9C%EB%8C%80_%EC%9C%A0%EB%9F%89
- https://cp-algorithms.com/graph/min_cost_flow.html#implementation

### 팁
- 무한루프가 걸린다면 V를 잘못 정했거나, 음의 사이클이 발생한 것이다.
  - 특히 V는 아무리 크게 잡더라도 오답을 내지 않으므로 예제입력을 다루다가 발견하지 못할 수 있다.
- 간선 연결, 특히 G의 연결 순서가 수행시간에 지대한 영향을 미친다. 
  - 이유 없이 TLE가 계속 난다면 src나 snk의 연결순서를 바꿔보자
  - 11405(책 구매하기) 참고